In [63]:
import numpy as np
from data_loader import load_dataset, load_challenge, save_challenge
from torch import nn
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from eval_script import compute_revenue

In [64]:
class BirdCrossEntropyLoss(nn.Module):
    def __init__(self, device):
        super(BirdCrossEntropyLoss, self).__init__()
        self.rev_matrix = torch.tensor(np.array(
            [[0.05, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2],
             [-0.25,  1., -0.3, -0.1, -0.1, -0.1, -0.1],
                [-0.02, -0.1,  1., -0.1, -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3,  1., -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1,  1., -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1,  1., -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1, -0.1,  1.]])).to(device)
        self.cet = nn.CrossEntropyLoss(reduction='none').to(device)


    def forward(self, input, target):
        ce_loss = self.cet(input, target)

        input_labels = torch.argmax(input, dim=1)
        target_labels = torch.argmax(target, dim=1)
        wanted = self.rev_matrix[target_labels, target_labels]
        actual = self.rev_matrix[target_labels, input_labels]
        cost = wanted - actual
        custom_loss = cost * ce_loss

        return custom_loss.mean()


In [65]:
class BirdDataset(Dataset):
    def __init__(self, birdset, is_train, device, scaler=None):
        self.device = device
        self.data = np.empty((0, 548), dtype=np.float32)
        self.labels = np.empty((0), dtype=int)
        for bird in birdset:
            if is_train:
                self.data = np.concatenate(
                    (self.data, np.concatenate(birdset[bird]['train_features'])))
                self.labels = np.concatenate(
                    (self.labels, np.concatenate(birdset[bird]['train_labels'])))
            else:
                self.data = np.concatenate(
                    (self.data, np.concatenate(birdset[bird]['test_features'])))
                self.labels = np.concatenate(
                    (self.labels, np.concatenate(birdset[bird]['test_labels'])))

        hot_matrix = np.eye(7)
        self.labels = hot_matrix[self.labels]

        if scaler == None:
            scaler = StandardScaler()
            scaler.fit(self.data)
        self.scaler = scaler

        self.data = torch.tensor(
            self.scaler.transform(self.data)).to(self.device)
        self.labels = torch.tensor(self.labels).to(self.device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        return x, y


In [66]:
def train(dataloader, model, loss_fn, optimizer, progress_steps):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if progress_steps != None and batch % progress_steps == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, show_progress):
    num_batches = len(dataloader)
    test_loss = 0
    money = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            money += compute_revenue(torch.argmax(pred, dim=1).cpu(), torch.argmax(y, dim=1).cpu())

    test_loss /= num_batches
    if show_progress:
        print(f"Avg loss: {test_loss:>8f}, Money saved: {money:.2f}$\n")
    return test_loss

def eval(model_fn, train_dataloader, test_dataloader, device, max_epochs):
    model = model_fn(None).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    loss_fn = BirdCrossEntropyLoss(device)
    lowest_test_loss = np.infty
    stop_criterion = 0
    for t in range(max_epochs):
        if t % 10 == 0:
            print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer, None)
        test_loss = test(test_dataloader, model, loss_fn, True)
        if test_loss > lowest_test_loss:
            stop_criterion += 1
        else:
            lowest_test_loss = test_loss
            stop_criterion = 0
        if stop_criterion >= 5:
            break
    print("Estimated performance:")
    test(test_dataloader, model, loss_fn, True)
    return model


In [67]:
model_fn = lambda _: nn.Sequential(
        nn.Linear(548, 256),
        nn.ReLU(),
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 7)
    )

In [68]:
dataset = load_dataset()

In [69]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
train_set = BirdDataset(dataset, True, device)
scaler = train_set.scaler
test_set = BirdDataset(dataset, False, device, scaler)
train_loader = DataLoader(train_set, batch_size=64)
test_loader = DataLoader(test_set, batch_size=64)

In [70]:
model = eval(model_fn, train_loader, test_loader, device, 500)

Epoch 1
-------------------------------
Avg loss: 0.651698, Money saved: 40.29$

Avg loss: 0.590067, Money saved: 278.74$

Avg loss: 0.533642, Money saved: 745.06$

Avg loss: 0.492740, Money saved: 1069.98$

Avg loss: 0.460804, Money saved: 1247.98$

Avg loss: 0.443339, Money saved: 1360.79$

Avg loss: 0.428384, Money saved: 1481.52$

Avg loss: 0.417607, Money saved: 1544.39$

Avg loss: 0.409060, Money saved: 1586.58$

Avg loss: 0.404205, Money saved: 1579.28$

Epoch 11
-------------------------------
Avg loss: 0.403769, Money saved: 1690.96$

Avg loss: 0.393066, Money saved: 1609.49$

Avg loss: 0.389124, Money saved: 1607.32$

Avg loss: 0.388686, Money saved: 1562.99$

Avg loss: 0.385223, Money saved: 1573.05$

Avg loss: 0.382511, Money saved: 1587.94$

Avg loss: 0.377780, Money saved: 1612.01$

Avg loss: 0.375792, Money saved: 1635.08$

Avg loss: 0.372444, Money saved: 1657.95$

Avg loss: 0.369766, Money saved: 1663.94$

Epoch 21
-------------------------------
Avg loss: 0.365146, Mo

In [74]:
torch.save(model.state_dict(), 'nn')

In [75]:
challenge = load_challenge()
print(challenge.shape)

(16, 3000, 548)


In [76]:
model = model_fn(None)
model.load_state_dict(torch.load('nn'))
model.eval()

model = model.to(device)
results = torch.Tensor(np.empty((challenge.shape[0], challenge.shape[1]))).to(device)
for i in range(challenge.shape[0]):
    challenge_subset = challenge[i]
    challenge_subset = scaler.transform(challenge_subset)
    challenge_subset = torch.Tensor(challenge_subset).unsqueeze(0).to(device)
    preds = model(challenge_subset).squeeze()
    pred_labels = torch.argmax(preds, dim=1)
    results[i, :] = pred_labels
results = results.int().cpu().numpy()
print(results, results.shape)
save_challenge('nn', results)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 3 3 3]
 ...
 [0 0 0 ... 5 0 0]
 [0 0 0 ... 0 6 6]
 [0 0 0 ... 0 0 0]] (16, 3000)
